# Import Liberaries

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import netCDF4 as nc
import xarray as xr
import geopandas as gpd

sys.path.append('/home/mahdi/WaterBalance/')

/home/mahdi/WaterBalance/myenv/lib/python3.8/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_5745/2886985012.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migrati

# Varables

In [7]:
PATH_DATA = "/mnt/e/Term2/payan_name/import/DataQDWB/"

# mashhad
# LAT_MIN = 35.80
# LAT_MAX = 37.11
# LON_MIN = 58.31
# LON_MAX = 60.14

# USA
LAT_MIN = 43.30
LAT_MAX = 44.60
LON_MIN = -109.40
LON_MAX = -107.40


# Functions

In [8]:
def mask_nc_file(
    nc,
    variable,
    lat_min,
    lat_max,
    lon_min,
    lon_max
):
    with xr.open_dataset(nc) as xr_nc:
        result = xr_nc.sel(
            lat = slice(lat_max, lat_min),
            lon = slice(lon_min, lon_max),
        )
    return result

# Load Data

In [9]:
# mm d-1 - precipitation
P = mask_nc_file(
    nc = PATH_DATA + "P_2022031.nc",
    variable = "precipitation",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)


# open the netCDF4 file
soil_moisture = nc.Dataset(PATH_DATA + "LPRM-AMSR2_L3_A_SOILM3_V001_20220131013015.nc4")

# print the variables in the dataset
print(soil_moisture)


mashhad = gpd.read_file('/mnt/e/Term2/WERI/test/Export_Output.shp')

usa = gpd.read_file('/mnt/e/Term2/payan_name/import/sample/usa.shp')


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    Conventions: CF-1.0
    dimensions(sizes): Longitude(1440), Latitude(720)
    variables(dimensions): float32 Latitude(Latitude), float32 Longitude(Longitude), int16 ts(Longitude, Latitude), int16 soil_moisture_c1(Longitude, Latitude), int16 opt_depth_c1(Longitude, Latitude), int16 soil_moisture_c2(Longitude, Latitude), int16 opt_depth_c2(Longitude, Latitude), int16 soil_moisture_x(Longitude, Latitude), int16 opt_depth_x(Longitude, Latitude), int16 mask(Longitude, Latitude), int16 soil_moisture_c1_error(Longitude, Latitude), int16 soil_moisture_c2_error(Longitude, Latitude), int16 soil_moisture_x_error(Longitude, Latitude), int16 frequency_map(Longitude, Latitude), float64 scantime(Longitude, Latitude)
    groups: 


In [10]:
from qdwb.primary_surface_flow.primary_surface_flow import *
from qdwb.primary_surface_flow.asset import *


R = np.array([PrimarySurfaceFlow.scs(
    precipitation = float(i),
    curve_number = 30,
    rsa = True,
    antecedent_precipitation = 100,
    is_growing_season = True
)[0]
for i in list(P.precipitation.values.flatten())])



underground_runoff = np.array([PrimarySurfaceFlow.scs(
    precipitation = float(i),
    curve_number = 30,
    rsa = True,
    antecedent_precipitation = 100,
    is_growing_season = True
)[1]
for i in list(P.precipitation.values.flatten())])


R_array = R.reshape(1,13,20)
underground_runoff_array = underground_runoff.reshape(1,13,20)

# array_dr = np.array([[[1,2,5], [3,4,6]]])
# array_dr.shape
ds_array = P.assign(Runoff=(['time', 'lat', 'lon'], R_array))
ds_array_1 = ds_array.assign(underground_runoff=(['time', 'lat', 'lon'], underground_runoff_array))
ds_array_1

<xarray.Dataset>
Dimensions:             (lon: 20, lat: 13, time: 1)
Coordinates:
  * lon                 (lon) float32 -109.3 -109.2 -109.1 ... -107.5 -107.4
  * lat                 (lat) float32 44.55 44.45 44.35 ... 43.55 43.45 43.35
  * time                (time) datetime64[ns] 2022-01-31
Data variables:
    precipitation       (time, lat, lon) float32 0.4375 0.4375 0.375 ... 0.0 0.0
    Runoff              (time, lat, lon) float64 12.64 12.64 ... 12.88 12.88
    underground_runoff  (time, lat, lon) int64 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
Attributes:
    history:          Created on 2022-02-05 07:15
    input_data_hash:  27d78e1ad0241438dcb7b796dee6d7c1393f1b4522680997748376e...

In [2]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [4]:
import geemap

Map = geemap.Map()
Map.add_basemap('HYBRID')

shape = ee.FeatureCollection('projects/ee-mohammadnejadmehdi77/assets/sample_usa')
# shape = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])
# LAT_MIN = 43.30
# LAT_MAX = 44.60
# LON_MIN = -109.40
# LON_MAX = -107.40

# Import the GCN250 Images and rename bands
# GCN250_Average =  ee.Image("users/jaafarhadi/GCN250/GCN250Average").select('b1').rename('average')
GCN250_Dry =  ee.Image("users/jaafarhadi/GCN250/GCN250Dry").select('b1').rename('dry')
# GCN250_Wet = ee.Image("users/jaafarhadi/GCN250/GCN250Wet").select('b1').rename('wet')

# visualize the Dry GCN dataset
# vis = {
#     min:40, 
#     max:75, 
#     palette: ['Red','SandyBrown','Yellow','LimeGreen', 'Blue','DarkBlue']
# }
# Crop Mask selection

CN_Wyoming = GCN250_Dry.clip(shape)
 
Map.centerObject(shape, 9)
Map.addLayer(CN_Wyoming, {'min':40, 'max':75, 'palette': ['Red','SandyBrown','Yellow','LimeGreen', 'Blue','DarkBlue']}, 'CN_Wyoming')
Map

Map(center=[43.88531410627492, -108.32657408653418], controls=(WidgetControl(options=['position', 'transparent…

In [12]:
array_CN = np.array(CN_Wyoming.toArray().getInfo()).astype(float)

R = np.array([PrimarySurfaceFlow.scs(
    precipitation = float(i),
    curve_number = array_CN,
    rsa = True,
    antecedent_precipitation = 100,
    is_growing_season = True
)[0]
for i in list(P.precipitation.values.flatten())])

EEException: Ok

In [ ]:
def scs_runoff(rainfall_depth, soil_group, land_use, slope):
    # Define curve number lookup table for different land uses
    cn_table = {
        'cropland': {'A': 65, 'B': 72, 'C': 79, 'D': 85},
        'pasture': {'A': 68, 'B': 76, 'C': 83, 'D': 89},
        'woodland': {'A': 30, 'B': 37, 'C': 47, 'D': 55},
        'urban': {'A': 85, 'B': 89, 'C': 92, 'D': 94},
    }
    
    # Define initial abstraction lookup table for different land uses
    ia_table = {
        'cropland': 0.2,
        'pasture': 0.2,
        'woodland': 0.1,
        'urban': 0.3,
    }
    
    # Calculate curve number based on hydrological soil group, land use, and slope
    cn = cn_table[land_use][soil_group] - (slope / 100.0) * 0.8 * (cn_table[land_use][soil_group] - 10)
    
    # Calculate initial abstraction based on land use and antecedent moisture conditions
    ia = ia_table[land_use] * cn
    
    # Calculate runoff depth based on rainfall depth, curve number, and initial abstraction
    runoff_depth = (rainfall_depth - ia) ** 2 / (rainfall_depth - ia + cn)
    
    return runoff_depth


rainfall_depth = 50  # mm
soil_group = 'B'
land_use = 'cropland'
slope = 5  # %

runoff_depth = scs_runoff(rainfall_depth, soil_group, land_use, slope)

print(f"Runoff depth: {runoff_depth:.2f} mm")


Runoff depth: 12.34 mm


In [46]:
import calendar
import os

import requests

this_dir = os.getcwd()
DATA_DIR = os.path.join(this_dir, 'data/L3_SM_P')

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)



def SMAP_L3_P_36km_Path(year, month, day):
    fpath_start = 'https://n5eil01u.ecs.nsidc.org/SMAP/'
    host = 'https://n5eil01u.ecs.nsidc.org/'
    version = '.004'
    url_path = '{host}/SMAP/SPL3SMP{version}/{year}.{month:02}.{day:02}/'.format(host=host,
                                                                                 version=version,
                                                                                 year=year,
                                                                                 month=month,
                                                                                 day=day)
    filename = 'SMAP_L3_SM_P_{year}{month:02}{day:02}_R14010_001.h5'.format(year=year, 
                                                                            month=month, 
                                                                            day=day)

    smap_data_path = url_path + filename
    
    return smap_data_path, filename


# Add Earthdata username and password here.
username = 'mahdi_mohammadnezhad77'
password = '12280967Mnm'

assert username and password, 'You must supply your Earthdata username and password!'


In [48]:
# Download data for March 2017
year = 2017
month = 3
_, days_in_month = calendar.monthrange(2017, 3)  # 31 days in March 2017

# Use a requests session to keep track of authentication credentials
with requests.Session() as session:
    session.auth = (username, password)
    
    for day in range(1, days_in_month + 1):
        print('Downloading SMAP data for: '+str(year)+'-'+str(month).zfill(2)+'-'+str(day).zfill(2))

        full_path, file_name = SMAP_L3_P_36km_Path(year, month, day)
        filepath = os.path.join(DATA_DIR, file_name)

        response = session.get(full_path)
        # If the response code is 401, we still need to authorize with earthdata.
        if response.status_code == 401:
            response = session.get(response.url)
        assert response.ok, 'Problem downloading data! Reason: {}'.format(response.reason)

        with open(filepath, 'wb') as f:
            f.write(response.content)

        print(file_name + ' downloaded')
        print('*** SM data saved to: '+ filepath +' *** ')

AssertionError: Problem downloading data! Reason: Not Found

In [40]:
import h5py
import matplotlib.pyplot as plt

# Open the H5 file in read-only mode
with h5py.File("/mnt/e/Term2/payan_name/import/soil_moisture/SMAP_L3_SM_P_20220131_R18290_001.h5", 'r') as file:
    # Access a dataset in the file
    # print(list(file.keys()))
    dataset = file['Soil_Moisture_Retrieval_Data_PM']


    # Get the group containing the members
    # group = dataset['soil_moisture_scah_pm']
    # Do something with the dataset
    print(list(dataset.keys()))
    # plt.imshow(group)
    # plt.colorbar()
    # plt.show()
    # print(dataset)
    


['EASE_column_index_pm', 'EASE_row_index_pm', 'albedo_dca_pm', 'albedo_pm', 'albedo_scah_pm', 'albedo_scav_pm', 'boresight_incidence_pm', 'bulk_density_pm', 'clay_fraction_pm', 'freeze_thaw_fraction_pm', 'grid_surface_status_pm', 'landcover_class_fraction_pm', 'landcover_class_pm', 'latitude_centroid_pm', 'latitude_pm', 'longitude_centroid_pm', 'longitude_pm', 'radar_water_body_fraction_pm', 'retrieval_qual_flag_dca_pm', 'retrieval_qual_flag_pm', 'retrieval_qual_flag_scah_pm', 'retrieval_qual_flag_scav_pm', 'roughness_coefficient_dca_pm', 'roughness_coefficient_pm', 'roughness_coefficient_scah_pm', 'roughness_coefficient_scav_pm', 'soil_moisture_dca_pm', 'soil_moisture_error_pm', 'soil_moisture_pm', 'soil_moisture_scah_pm', 'soil_moisture_scav_pm', 'static_water_body_fraction_pm', 'surface_flag_pm', 'surface_temperature_pm', 'surface_water_fraction_mb_h_pm', 'surface_water_fraction_mb_v_pm', 'tb_3_corrected_pm', 'tb_4_corrected_pm', 'tb_h_corrected_pm', 'tb_h_uncorrected_pm', 'tb_qual_